In [2]:
import numpy as np
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
def data_loader(data_dir, batch_size, random_seed=42, val_size=0.1, shuffle=True, test=False):
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010]
    )

    transform = transforms.Compose([
        transforms.Resize((227, 227)),
        transforms.ToTensor(),
        normalize
    ])

    if test:
        dataset = datasets.CIFAR100(
            root=data_dir, train=False,
            download=True, transform=transform
        )

        dataloader = torch.utils.data.DataLoader(
            dataset, shuffle=shuffle, batch_size=batch_size
        )
        return dataloader

    train_dataset = datasets.CIFAR100(
        root=data_dir, train=True,
        download=True, transform=transform
    )
    val_dataset = datasets.CIFAR100(
        root=data_dir, train=True,
        download=True, transform=transform
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(val_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, val_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler
    )

    val_dataloader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, sampler=val_sampler
    )

    return (train_dataloader, val_dataloader)


train_dataloader, val_dataloader = data_loader(data_dir="./data", batch_size=64)
test_dataloader = data_loader(data_dir="./data", batch_size=64, shuffle=False, test=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
len(train_dataloader), len(val_dataloader), len(test_dataloader)

(704, 79, 157)

In [6]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()

        self.convolutions = nn.Sequential(
            # 1st layer
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # 2nd layer
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 3rd layer
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # 4th layer
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 5th layer
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            # 6th layer
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            # 7th layer
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 8th layer
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 9th layer
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 10th layer
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 11th layer
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 12th layer
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 13th layer
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Flatlining, oh, I wanted to say flattening
            nn.Flatten()
        )

        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, X):
        return self.fc(self.convolutions(X))

In [7]:
num_classes = 100
num_epochs = 20
learning_rate = 0.001

model = VGG16(num_classes).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.005, momentum=0.9)

total_step = len(train_dataloader)

In [8]:
!pip install torchmetrics

In [9]:
from torchmetrics import Accuracy

In [10]:
train_step = len(train_dataloader)
val_step = len(val_dataloader)
accuracy_fn = Accuracy(task="multiclass", num_classes=100).to(device)

for epoch in range(num_epochs):
    # Train step
    model.train()
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        outputs = torch.softmax(logits, dim=1).argmax(dim=1)
        loss = loss_fn(logits, labels)
        accuracy = accuracy_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train| Step {i+1}/{train_step}, Loss: {loss.item():.4f}")

    # Validation step
    model.eval()
    with torch.inference_mode():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(val_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            logits = model(images)
            _, predicted = torch.max(logits.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, logits

        print(f"Test| Step {i+1}/{val_step}, Loss: {loss.item():.4f}, Accuracy: {100*correct/total:.4f}")
        print("="*60)

Epoch 1/20
Train| Step 704/704, Loss: 3.1800
Test| Step 79/79, Loss: 3.1800, Accuracy: 14.7200
Epoch 2/20
Train| Step 704/704, Loss: 2.9763
Test| Step 79/79, Loss: 2.9763, Accuracy: 24.4000
Epoch 3/20
Train| Step 704/704, Loss: 2.8542
Test| Step 79/79, Loss: 2.8542, Accuracy: 28.7200
Epoch 4/20
Train| Step 704/704, Loss: 2.2286
Test| Step 79/79, Loss: 2.2286, Accuracy: 36.9000
Epoch 5/20
Train| Step 704/704, Loss: 1.5869
Test| Step 79/79, Loss: 1.5869, Accuracy: 43.2000
Epoch 6/20
Train| Step 704/704, Loss: 1.9646
Test| Step 79/79, Loss: 1.9646, Accuracy: 45.4000
Epoch 7/20
Train| Step 704/704, Loss: 2.4628
Test| Step 79/79, Loss: 2.4628, Accuracy: 47.7800
Epoch 8/20
Train| Step 704/704, Loss: 0.8611
Test| Step 79/79, Loss: 0.8611, Accuracy: 47.6000
Epoch 9/20
Train| Step 704/704, Loss: 1.6571
Test| Step 79/79, Loss: 1.6571, Accuracy: 52.4200
Epoch 10/20
Train| Step 704/704, Loss: 1.1961
Test| Step 79/79, Loss: 1.1961, Accuracy: 54.1200
Epoch 11/20
Train| Step 704/704, Loss: 1.0580
Tes